In [63]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# The overview of data retrieved from arxiv 

In [64]:
import arxiv

In [ ]:
dir(arxiv)

In [ ]:
print(arxiv.query(search_query="au: Dieter Horns"))

Table: search_query field prefixes
* prefix	explanation
* ti	Title
* au	Author
* abs	Abstract
* co	Comment
* jr	Journal Reference
* cat	Subject Category
* rn	Report Number
* id	Id (use id_list instead)
* all	All of the above

Search the result, access and download. Tidy up the name of the files

In [ ]:
import arxiv
# Query for a paper of interest, then download
paper = arxiv.query(id_list=["1707.08567"])[0]
arxiv.download(paper)
# You can skip the query step if you have the paper info!
paper2 = {"pdf_url": "http://arxiv.org/pdf/1707.08567v1",
          "title": "The Paper Title"}
arxiv.download(paper2)

# Returns the object id
def custom_slugify(obj):
    return obj.get('id').split('/')[-1]

# Download with a specified slugifier function
arxiv.download(paper, slugify=custom_slugify)

In [ ]:
paper.get('title')
paper.get('authors')
paper.get('summary')

In [52]:
%ls

1707.08567v1.Proceedings_of_Workshop_AEW10_Concepts_in_Information_Theory_and_Communications.pdf
1707.08567v1.The_Paper_Title.pdf
1707.08567v1.pdf
Untitled.ipynb
cta_data_analysis.ipynb
fermi_lat.ipynb


In [ ]:
paper = arxiv.query(id_list=["1707.08567"])[0]

# Retrieve the data of an author 

In [51]:
DH = arxiv.query(search_query="au: Dieter Horns")
DH[0]

{'id': 'http://arxiv.org/abs/1309.3846v1',
 'guidislink': True,
 'updated': '2013-09-16T08:08:39Z',
 'updated_parsed': time.struct_time(tm_year=2013, tm_mon=9, tm_mday=16, tm_hour=8, tm_min=8, tm_sec=39, tm_wday=0, tm_yday=259, tm_isdst=0),
 'published': '2013-09-16T08:08:39Z',
 'published_parsed': time.struct_time(tm_year=2013, tm_mon=9, tm_mday=16, tm_hour=8, tm_min=8, tm_sec=39, tm_wday=0, tm_yday=259, tm_isdst=0),
 'title': 'Pair-production opacity at high and very-high gamma-ray energies',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'http://export.arxiv.org/api/query?search_query=au%3A+Dieter+Horns&id_list=&start=0&max_results=10&sortBy=relevance&sortOrder=descending',
  'value': 'Pair-production opacity at high and very-high gamma-ray energies'},
 'summary': 'The propagation of high energy (HE, $E_\\gamma>100$ MeV) and very high-energy\ngamma-rays (VHE, $E_\\gamma>100$ GeV) in the extra-galactic photon field leads\nto pair-production and consequently ene

In [ ]:
DH[9]['title']
DH[11]

In [ ]:
DH['title']

In [ ]:
DH[1] ['title']

In [ ]:
for DHevery in DH: 
    print(DHevery['title'])
    print(DHevery['authors'])
    #print(DHevery['summary'])

In [ ]:
DHpaper = arxiv.query(id_list= ["1309.3846"])[0]

In [55]:
DH = arxiv.query(search_query="au: Dieter Horns", max_results= 50)

In [ ]:
for DHevery in DH: 
    print(DHevery['title'])
    print(DHevery['authors'])
    #print(DHevery['summary'])

In [ ]:
paper2 = {"pdf_url": "http://arxiv.org/pdf/1707.08567v1",
          "title": "The Paper Title"}
arxiv.download(paper2)

In [ ]:
for DHevery in DH: 
    everyitem = {"pdf_url": DHevery["pdf_url"],
                 "title" : DHevery["title"]}
    arxiv.download(everyitem)
    

In [ ]:
!touch DH_index.md

In [78]:
DH = arxiv.query(search_query="au: Dieter Horns", max_results= 300)

In [ ]:
with open('DH_index.md', 'w') as the_file:
    for DHevery in DH: 
        the_file.write('* **Title:** ' + DHevery["title"] + '\n')  
        the_file.write('\n')
        the_file.write('  **Published at:** ' + DHevery['published'] + '\n')
        the_file.write('\n')
        the_file.write('  **pdf_url:** ' + DHevery['pdf_url'] + '\n') 
        the_file.write('\n')
        the_file.write('  **Summary:**' + DHevery["summary"] + '\n') 
    the_file.close()

## Workflow

Then you can draw the content of summary from the retrieved data, then read the summary and find the interesting paper and download it with the help of its `pdf_url`.

## How to determine the author is the only one

## Store the data using pandas